In [ ]:
import pandas as pd
import os
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.dates as md
import datetime as dt
from pivottablejs import pivot_ui
from IPython.display import HTML
import seaborn as sns
import rpy2
import pickle
import matplotlib as mp
from IPython.display import display, HTML
from PIL import Image
from scipy import stats as st
from scipy.stats import t
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
from dask.distributed import Client
from dask.distributed import as_completed
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.display.max_columns = 50
pd.options.display.max_rows = 200
#import qgrid
#import ipywidgets as widgets

In [ ]:
def time_to_int(dateobj):
    total = int(dateobj.strftime('%S'))
    total += int(dateobj.strftime('%M')) * 60
    total += int(dateobj.strftime('%H')) * 60 * 60
    total += (int(dateobj.strftime('%j')) - 1) * 60 * 60 * 24
    total += (int(dateobj.strftime('%Y')) - 1970) * 60 * 60 * 24 * 365
    return total

def myfunc(x):
    return slope * x + intercept

In [ ]:
import numpy as np

class expanding_window(object):
    '''	
    Parameters 
    ----------
    
    Note that if you define a horizon that is too far, then subsequently the split will ignore horizon length 
    such that there is validation data left. This similar to Prof Rob hyndman's TsCv 
    
    
    initial: int
        initial train length 
    horizon: int 
        forecast horizon (forecast length). Default = 1
    period: int 
        length of train data to add each iteration 
    '''
    

    def __init__(self,initial= 1,horizon = 1,period = 1):
        self.initial = initial
        self.horizon = horizon 
        self.period = period 


    def split(self,data):
        '''
        Parameters 
        ----------
        
        Data: Training data 
        
        Returns 
        -------
        train_index ,test_index: 
            index for train and valid set similar to sklearn model selection
        '''
        self.data = data
        self.counter = 0 # for us to iterate and track later 


        data_length = data.shape[0] # rows 
        data_index = list(np.arange(data_length))
         
        output_train = []
        output_test = []
        # append initial 
        output_train.append(list(np.arange(self.initial)))
        progress = [x for x in data_index if x not in list(np.arange(self.initial)) ] # indexes left to append to train 
        output_test.append([x for x in data_index if x not in output_train[self.counter]][:self.horizon] )
        # clip initial indexes from progress since that is what we are left 
         
        while len(progress) != 0:
            temp = progress[:self.period]
            to_add = output_train[self.counter] + temp
            # update the train index 
            output_train.append(to_add)
            # increment counter 
            self.counter +=1 
            # then we update the test index 
            
            to_add_test = [x for x in data_index if x not in output_train[self.counter] ][:self.horizon]
            output_test.append(to_add_test)

            # update progress 
            progress = [x for x in data_index if x not in output_train[self.counter]]	
            
        # clip the last element of output_train and output_test
        output_train = output_train[:-1]
        output_test = output_test[:-1]
        
        # mimic sklearn output 
        index_output = [(train,test) for train,test in zip(output_train,output_test)]
        
        return index_output

In [ ]:
def evaluate_ets(npa_):
    train = npa_[0]
    test = npa_[1]
    
    ets_TT=sm.tsa.statespace.ExponentialSmoothing(np.log(train),initialization_method='heuristic',seasonal=52,trend=True,damped_trend=True).fit()
    ets_FT=sm.tsa.statespace.ExponentialSmoothing(np.log(train),initialization_method='heuristic',seasonal=52,trend=False,damped_trend=True).fit()
    ets_TF=sm.tsa.statespace.ExponentialSmoothing(np.log(train),initialization_method='heuristic',seasonal=52,trend=True,damped_trend=False).fit()
    ets_FF=sm.tsa.statespace.ExponentialSmoothing(np.log(train),initialization_method='heuristic',seasonal=52,trend=False,damped_trend=False).fit()
    #arima_model = auto_arima(train, start_p=0, start_d=0, start_q=0,max_p=3, max_d=3, max_q=5, start_P=0, start_D=0, start_Q=0, max_P=3, max_D=3, max_Q=3, m=52, seasonal=True, stepwise=True, n_fits=50)

    #labels_ = list()
    fc_ets_TT = np.exp(ets_TT.forecast(13)) #inverting the Log
    fc_ets_FT = np.exp(ets_FT.forecast(13)) #inverting the Log
    fc_ets_TF = np.exp(ets_TF.forecast(13)) #inverting the Log
    fc_ets_FF = np.exp(ets_FF.forecast(13)) #inverting the Log

    rmse_TT = mean_squared_error(test, fc_ets_TT, squared=True)
    rmse_FT = mean_squared_error(test, fc_ets_FT, squared=True)
    rmse_TF = mean_squared_error(test, fc_ets_TF, squared=True)
    rmse_FF = mean_squared_error(test, fc_ets_FF, squared=True)
    return([rmse_TT, rmse_FT, rmse_TF, rmse_FF])

In [ ]:
db_filename = 'todo.db'

db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print('Need to create schema')
else:
    print('Database exists, assume schema does, too.')

conn.close()

In [ ]:
si = ['03-31','06-30','09-30','12-31']
               
[start,end, prices_df, sp1500_index_df, sp500, sp600, sp400, market_data, completed_fred_pvt, completed_bonds, completed_bonds_pvt, sectors, indexes, screener_sorted] = pickle.load(open('./data_object.pkl', 'rb'))

prices = prices_df
#prices = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="SP1500_Sectors_Indexes_Prices")

In [ ]:
def colored(r, g, b, text):
    return f"\033[38;2;{r};{g};{b}m{text}\033[0m"

In [ ]:
def display_side_by_side(dfs:list, captions:list, tablespacing=5):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    for (caption, df) in zip(captions, dfs):
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += tablespacing * "\xa0"
    display(HTML(output))

In [ ]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start, end_date=end)
"""
pd.to_datetime(idx2, errors='coerce')
df.index = pd.to_datetime(df.index, errors='coerce',format='%Y-%m-%d')
df = df.interpolate(method='time')

frequency = nyse_trading_dates.rename_axis([None]).index

df = prices.rename_axis([None]).reindex(frequency)

for d in prices.rename_axis([None]).index:
    df.loc[d] = prices.loc[d]

df.interpolate(method='linear')

#nyse_trading_dates.reset_index().set_index('index').rename_axis([None]).index
"""

In [ ]:
#stock_fundamentals = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="Stock_Fundamentals")
stock_fundamentals = market_data

In [ ]:
#market_data

In [ ]:
#print(stock_fundamentals)

In [ ]:

import warnings
warnings.filterwarnings("ignore")

# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error


# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    

# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)


In [ ]:
#stock_fundamentals[stock_fundamentals.set_index(stock_fundamentals.iloc[:, 0]).index=='VOX']


In [ ]:
#prices[prices['Symbol']=='VOX'].set_index('Date')

In [ ]:
#plt.plot(prices[prices['Symbol']=='VOX'].set_index('Date')['Adj Close'])

In [ ]:
#screener = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="Screener")
screener = screener_sorted


In [ ]:
#screener.set_index('Symbol').loc['BA']

In [ ]:
#fundamentals = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="Stock_Fundamentals")
#fundamentals.set_index(fundamentals.columns[0],inplace=True)
fundamentals = stock_fundamentals

In [ ]:
"""
stock_indexes = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="SP1500")
stock_indexes.drop(stock_indexes.columns[0], axis=1, inplace=True)
stock_indexes.set_index(stock_indexes.columns[0],inplace=True)
stock_indexes
"""
stock_indexes = sp1500_index_df

In [ ]:
"""
spdr_indexes = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="spdr_indexes")
#vanguard_sectors.drop(vanguard_sectors.columns[0], axis=1, inplace=True)
spdr_indexes.set_index(spdr_indexes.columns[0],inplace=True)
spdr_indexes.columns=['Symbol']
spdr_indexes
"""
spdr_indexes = indexes
spdr_indexes.columns=['Symbol']
#spdr_indexes

In [ ]:
"""
vanguard_sectors = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="vanguard_sectors")
#vanguard_sectors.drop(vanguard_sectors.columns[0], axis=1, inplace=True)
vanguard_sectors.set_index(vanguard_sectors.columns[0],inplace=True)
vanguard_sectors.columns=['Symbol']
vanguard_sectors
"""
vanguard_sectors = sectors
vanguard_sectors.columns=['Symbol']
#vanguard_sectors

In [ ]:
sector_performance_return = pd.DataFrame()
sector_performance_Q_smoothed_return = pd.DataFrame()

for v in [*spdr_indexes['Symbol'],*vanguard_sectors['Symbol']]:
    subset = prices[prices['Symbol'] == v]
    #subset_ = subset.set_index('Date').asfreq('Q')[['Adj Close']].pct_change()
    subset_ = subset.asfreq('D').interpolate().asfreq('Q')[['Adj Close']].pct_change()
    #print(subset_)
    #subset_qs = subset.set_index('Date')[['Adj Close']].pct_change().rolling(63).mean()
    subset_qs = subset[['Adj Close']].pct_change().rolling(63).mean()
    subset_.index = subset_.index.values.astype('M8[D]')
    subset_qs.index = subset_qs.index.values.astype('M8[D]')
    dt_str = subset_.index[-1].strftime('%Y-%m-%d')
    #tail = subset[subset.set_index('Date').index>=dt_str]
    tail = subset[subset.index>=dt_str]
    #dt_str = subset.set_index('Date').index[-1].strftime('%Y-%m-%d')
    dt_str = subset.index[-1].strftime('%Y-%m-%d')
    #tail_ = pd.DataFrame([(tail.set_index('Date')['Adj Close'][-1]-tail.set_index('Date')['Adj Close'][0])/tail.set_index('Date')['Adj Close'][0]],index=[datetime.strptime(dt_str, '%Y-%m-%d')],columns=['Adj Close'])
    tail_ = pd.DataFrame([(tail['Adj Close'][-1]-tail['Adj Close'][0])/tail['Adj Close'][0]],index=[datetime.strptime(dt_str, '%Y-%m-%d')],columns=['Adj Close'])
    subset__ = pd.concat([subset_,tail_],axis=0)
    sector_performance_return = pd.concat([sector_performance_return,subset__],axis=1)
    sector_performance_Q_smoothed_return = pd.concat([sector_performance_Q_smoothed_return,subset_qs],axis=1)

In [ ]:
sector_performance_Q_smoothed_return.columns = [*[i for i in spdr_indexes.index],*[v for v in vanguard_sectors.index]]
sector_performance_return.columns = [*[i for i in spdr_indexes.index],*[v for v in vanguard_sectors.index]]
#display(sector_performance_Q_smoothed_return)
#display(sector_performance_return)

In [ ]:
cm = sns.color_palette("blend:red,yellow,green", as_cmap=True)
#pd.crosstab(shares_viz['Year'], shares_viz['Month'])

std_indexes = pd.DataFrame()
seasonal_indexes = pd.DataFrame()

for spr in sector_performance_return.columns:
    #print(spr)
    s = sector_performance_return[[spr]]
    #print(s)
    years = []
    quarters = []
    for d in sector_performance_return[spr].index:

        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)

    s['Years'] = years
    s['Quarters'] = quarters
    ct = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).mean(axis=0).sort_index()
    ct.columns = [spr]
    
    ct_std = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).std(axis=0).sort_index()
    ct_std.columns = [spr]
    
    std_indexes = pd.concat([std_indexes,ct_std],axis=1)
    seasonal_indexes = pd.concat([seasonal_indexes,ct],axis=1)

seasonal_indexes.columns = sector_performance_return.columns

std_indexes.columns = sector_performance_return.columns
    
seasonal_indexes = seasonal_indexes.T

std_indexes = std_indexes.T

si = ['03-31','06-30','09-30','12-31']

print("Seasonal Indexes")
"""
def b_g(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    #print(s)
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    return ['background-color: %s' % color for color in c]
"""

def b_g(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    return ['background-color: %s' % color for color in c]
seasonal_index_values_sorted = []
seasonal_index_names_sorted = []
#B = seasonal_indexes[si]
for c in seasonal_indexes[si].columns:
    seasonal_index_values_sorted.append(seasonal_indexes[si][c].sort_values(ascending=False).values)
    seasonal_index_names_sorted.append(seasonal_indexes[si][c].sort_values(ascending=False).index.values)

temp_seasonal_index_values_sorted = pd.DataFrame(seasonal_index_values_sorted).T
temp_seasonal_index_values_sorted.columns = seasonal_indexes[si].columns

temp_seasonal_index_names_sorted = pd.DataFrame(seasonal_index_names_sorted).T
temp_seasonal_index_names_sorted.columns = seasonal_indexes[si].columns

A = temp_seasonal_index_values_sorted
B = temp_seasonal_index_names_sorted 
#B.columns = A.columns

display(B.style.apply(b_g,cmap=cm))

#A = seasonal_indexes[si]
#B = 
#display(df)

display(seasonal_indexes[si].style.background_gradient(cmap = cm,axis=0))

print("Note if the final data in the prices ends on a quarter, this will aggregate into the seasonal_index above and there will be no comparison")
print("Current")

#A = seasonal_indexes[seasonal_indexes.columns.difference(si)].sort_values(by=seasonal_indexes[seasonal_indexes.columns.difference(si)].columns[0],ascending=False)
#B = seasonal_indexes[seasonal_indexes.columns.difference(si)].sort_values(by=seasonal_indexes[seasonal_indexes.columns.difference(si)].columns[0],ascending=False)
#display(B.style.apply(b_g,cmap=cm))
display(seasonal_indexes[seasonal_indexes.columns.difference(si)].sort_values(by=seasonal_indexes[seasonal_indexes.columns.difference(si)].columns[0],ascending=False).style.background_gradient(cmap = cm,axis=0))    

#calculate p-value
n=5
#print("means")
#display(seasonal_indexes[si])
print("standard deviations")
display(std_indexes[si])
t_scores = (seasonal_indexes[si])/(std_indexes[si]*np.sqrt(n))
print("t scores")
display(t_scores)
print("p values")
p_values = pd.DataFrame(t.cdf(t_scores, df=n-2),index=seasonal_indexes[si].index,columns=seasonal_indexes[si].columns)
def highlight_cells(val):
    color = 'yellow' if val <= .50 else 'green'
    #color = 'green' if val > .50 else ''
    return 'background-color: {}'.format(color)

display(p_values.style.applymap(highlight_cells))

p_values.hist()

#st.ttest_1samp(seasonal_indexes[si]/std_indexes[si],popmean=0,df=4-2)

#Ttest_1sampResult(statistic=0.36789006583267403, pvalue=0.714539654336473)

In [ ]:
cycles_ = ['Expansion','Slowdown','Recession','Recovery']

path=r"C:\Users\User\Documents\wiki\wiki\dev\python\Python-Stock\reports\figures\Business cycles.png"
display(Image.open(path))
path=r"C:\Users\User\Documents\wiki\wiki\Finance\Business Cycles-2.png"
display(Image.open(path))

In [ ]:
from scipy import stats as st
from scipy.stats import t

sector_performance_return_pvt = sector_performance_return[sector_performance_return.iloc[-1].sort_values(ascending=False).index]

sector_performance_return_pvt_heatmap = sector_performance_return_pvt.style.background_gradient(cmap = cm,axis=None)
display(sector_performance_return_pvt_heatmap)

sector_performances = pd.DataFrame()
sector_performances_values = pd.DataFrame()
for c in sector_performance_return.T.columns:
    #print(c)
    temp = pd.DataFrame(sector_performance_return.T[c].sort_values(ascending=False).index,columns=[c])
    #print(temp)
    temp2 = pd.DataFrame(sector_performance_return.T[c].sort_values(ascending=False),columns=[c])
    #print(temp2)
    temp2.reset_index(drop=True,inplace=True)    
    sector_performances = pd.concat([sector_performances,temp],axis=1)
    sector_performances_values = pd.concat([sector_performances_values,temp2],axis=1)
    #print(sector_performance_return[c].sort_values(ascending=False))
#display(sector_performances_values)
#sector_performances

def b_gfs(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    
    colors = [color for color in c]
    strings = []
    """
    for c in range(0,len(colors)):
        c_ = colors[c]
        strings.append(
    
    
    return **{
            'background-color': 'grey',
            'font-size': '20pt',
        }
    """
    return([*np.repeat('font-size: 7pt', len(colors), axis = None)])
    #print(['background-color: %s' % color for color in c])
    #print(['background-color: %s' % color + ', font-size: 7pt' for color in c])
    #return ['background-color: %s' % color for color in c]
    #return ['background-color: %s' % color + ', font-size: 7pt' for color in c]
    #return ['background-color: %s' % color for color in c,'font-size': '7pt']


def b_g(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    
    return ['background-color: %s' % color for color in c]
    
from pandas.io.formats.style import Styler
    
A = sector_performances_values
B = sector_performances

df = B.style.apply(b_g,cmap='RdYlGn')
#print(type(df.export()))
"""
df_styler_1 = df.export()
df_styler_2 = B.style.apply(b_g,cmap='RdYlGn').export()
Styler.use([df_styler_1])    
Styler.use([df_styler_2])
"""


#df.index = A.index
#sector_performances.style.background_gradient(cmap = cm,axis=1,)

display(df)


In [ ]:
#cm = sns.color_palette("blend:red,yellow,green", as_cmap=True)
#pd.crosstab(shares_viz['Year'], shares_viz['Month'])

#seasonal_indexes[si]
seasonally_adjusted = pd.DataFrame()

for s in sector_performance_return.columns:
    
    s_ = pd.DataFrame(sector_performance_return[s])
    #print(sector_performance_return[s])
    years = []
    quarters = []
    
    for d in s_.index:
        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)
        
    s_['Quarter'] = quarters
    si_ = seasonal_indexes.loc[s]
    
    modified = pd.DataFrame()
    #print(si_.index)
    for s_i_ in si_.index:
        #print(s_i_)
        #when it finds the last date (which hopefully isn't a quarter date)
        if(np.sum(np.array(si)==s_i_)==0):
            #print("don't modify")
            #print(s_i_)
            match = []
            for entry in si:
                if(int(s_i_.rsplit("-")[0])<=int(entry.rsplit("-")[0])):
                    match = entry
                    break            

            delta_months = int(match.rsplit("-")[0])-int(s_i_.rsplit("-")[0])
            delta_days_ = int(match.rsplit("-")[1])-int(s_i_.rsplit("-")[1])
            
            delta_days = int(delta_months*(91.25/3)+delta_days_)
            #daily interest
            
            #print(delta_days)
            #print(seasonal_indexes.loc[s].loc[match])
            seasonal_index_to_daily = (1+seasonal_indexes.loc[s].loc[match])**(1/91.25)-1
            #print("Seasonal Index to Daily Discounted Interest Rate:",seasonal_index_to_daily)
            daily_extrapolated = (1+seasonal_index_to_daily)**delta_days-1
            #print(daily_extrapolated)
            #print("Daily ^ delta_days:",daily_extrapolated)
            
            #print(match)
            #print(s_i_)
            seasonally_adjusted_column = pd.DataFrame([seasonal_indexes.loc[s].loc[match] - daily_extrapolated],columns=[s],index=[s_.index[-1]])
            
            
        else:
            seasonally_adjusted_column = pd.DataFrame(s_[s][s_['Quarter']==s_i_] - seasonal_indexes.loc[s].loc[s_i_],columns=[s])
        modified = pd.concat([modified,seasonally_adjusted_column],axis=0)
        modified.sort_index(inplace=True)
    #print(modified)
    seasonally_adjusted = pd.concat([seasonally_adjusted,modified],axis=1)

print("Comparative to Seasonal Indexes, tells you who is over or underperforming.")
print("This is Seasonal Index - Extrapolated Current Quarter.")
print("+: seasonal > current")
print("-: current > seasonal")

print("Note if the final data in the prices ends on a quarter, this will aggregate into the seasonal_index above and there will be no comparison")
    
    
std_indexes_seas_adj = pd.DataFrame()
seasonal_indexes_seas_adj = pd.DataFrame()

temp_ = pd.DataFrame(seasonally_adjusted.T.iloc[:,-1])
temp_.columns = pd.DataFrame(seasonally_adjusted.T.iloc[:,-1]).columns.strftime('%Y-%m-%d')
display(temp_.sort_values(by=temp_.columns[0],ascending=False).style.background_gradient(cmap = cm,axis=0))

#display(seasonally_adjusted[seasonally_adjusted.columns.difference(si)].style.background_gradient(cmap = cm,axis=0))

for spr in seasonally_adjusted.columns:
    #print(spr)
    s = seasonally_adjusted[[spr]]
    #print(s)
    years = []
    quarters = []
    for d in seasonally_adjusted[spr].index:
        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)

    s['Years'] = years
    s['Quarters'] = quarters
    ct = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).mean(axis=0).sort_index()
    ct.columns = [spr]
    #print(ct)
    
    ct_std = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).std(axis=0).sort_index()
    ct_std.columns = [spr]
    
    std_indexes_seas_adj = pd.concat([std_indexes_seas_adj,ct_std],axis=1)
    seasonal_indexes_seas_adj = pd.concat([seasonal_indexes_seas_adj,ct],axis=1)

seasonal_indexes_seas_adj.columns = seasonally_adjusted.columns

std_indexes_seas_adj.columns = seasonally_adjusted.columns
    
seasonal_indexes_seas_adj = seasonal_indexes_seas_adj.T

std_indexes_seas_adj = std_indexes_seas_adj.T

sector_performances_sea_adj = pd.DataFrame()
sector_performances_values_sea_adj = pd.DataFrame()
for c in seasonally_adjusted.T.columns:
    #print(c)
    temp = pd.DataFrame(seasonally_adjusted.T[c].sort_values(ascending=False).index,columns=[c])
    #print(temp)
    temp2 = pd.DataFrame(seasonally_adjusted.T[c].sort_values(ascending=False),columns=[c])
    #print(temp2)
    temp2.reset_index(drop=True,inplace=True)    
    sector_performances_sea_adj = pd.concat([sector_performances_sea_adj,temp],axis=1)
    sector_performances_values_sea_adj = pd.concat([sector_performances_values_sea_adj,temp2],axis=1)
    #print(sector_performance_return[c].sort_values(ascending=False))
#display(sector_performances_values)
#sector_performances

def b_g(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    return ['background-color: %s' % color for color in c]

A = sector_performances_values_sea_adj
B = sector_performances_sea_adj

df_sea_adj = B.style.apply(b_g,cmap='RdYlGn')
#df.index = A.index
#sector_performances.style.background_gradient(cmap = cm,axis=1,)
display(df_sea_adj)
#display(seasonally_adjusted[seasonally_adjusted.columns.difference(si)].style.background_gradient(cmap = cm,axis=0))


In [ ]:
#seasonally_adjusted

In [ ]:
index_performance = []
for v in spdr_indexes['Symbol'].values:
    #print(v)
    index_performance.append([v,screener.loc[v]['risk_trend_factor']])
    #print([v,)
temp_i = pd.DataFrame(index_performance,index=spdr_indexes.index)
temp_i.columns = ['Symbol','risk_trend_factor']
index_risk_sorted = pd.DataFrame(index_performance,columns=['Symbol','risk_trend_factor'],index=spdr_indexes.index).sort_values(by='risk_trend_factor',ascending=False).reset_index()
index_risk_sorted.columns = ['Index','Symbol','risk_trend_factor']
index_risk_sorted

In [ ]:
sector_performance = []
for v in vanguard_sectors['Symbol'].values:
    #print(v)
    sector_performance.append([v,screener.loc[v]['risk_trend_factor']])
    #print([v,)
temp_v = vanguard_sectors.reset_index()
temp_v.columns = ['Sector','Symbol']
sectors_risk_sorted = pd.DataFrame(sector_performance,columns=['Symbol','risk_trend_factor']).merge(temp_v, on='Symbol', how='left').sort_values(by='risk_trend_factor',ascending=False)    
sectors_risk_sorted


In [ ]:
print("Rolling Quarterly Mean Return")
plt.plot(sector_performance_Q_smoothed_return)
plt.legend([*[i for i in spdr_indexes.index],*[v for v in vanguard_sectors.index]],loc=2,fontsize=7)
plt.show()

In [ ]:
print ("Quarterly Returns")
for v in sector_performance_return.T.index:
    subset = pd.DataFrame(sector_performance_return.T.loc[v])
    subset.columns = ['Return']
    plt.plot(subset)
    plt.xticks(subset.index, rotation=45)
plt.legend(sector_performance_return.T.index,loc=2)    
plt.show()


In [ ]:
#sector_performance_return.iloc[-1].sort_values(ascending=False).index

In [ ]:
#sector_performance_return

In [ ]:
#seasonal_indexes.index

In [ ]:
plt.plot(seasonal_indexes[si].T)
plt.legend(seasonal_indexes[si].T.columns,loc=2,fontsize=8)
plt.show()

In [ ]:
#seasonal_indexes[seasonal_indexes.columns.difference(si)]

In [ ]:
threshold = .975

In [ ]:
screener_sorted_ = screener_sorted.reset_index().merge(stock_indexes,on='Symbol',how='left')


screener_good = screener_sorted_[screener_sorted_['risk_trend_factor']!='missing']

risk_trend_threshold = screener_good['risk_trend_factor'].quantile(q=[threshold]).values[0]


In [ ]:
risk_trend_threshold

In [ ]:
stock_sectors = stock_fundamentals['sector']

In [ ]:
temp = stock_sectors.reset_index()
temp.columns = ['Symbol','Sector']
sector_merged = screener[['risk_trend_factor']].merge(temp, on='Symbol', how='right')
temp_sm = sector_merged.merge(sp1500_index_df, on='Symbol', how='left').set_index('Symbol')
temp_sm.columns = ['risk_trend_factor', 'Sector', 'index']
temp_sm = temp_sm.merge(spdr_indexes.reset_index(), on='index', how='left')
temp_sm.columns = ['risk_trend_factor', 'Sector', 'index','index_Symbol']
temp_sm.index = sector_merged['Symbol']

for spdr_ in range(0,len(spdr_indexes['Symbol'].values)):
    spdr__ = spdr_indexes['Symbol'].values[spdr_]
    #print(spdr__)
    spdr__i = spdr_indexes['Symbol'].index[spdr_]
    #print(spdr__i)
    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index']] = spdr__i
    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index_Symbol']] = spdr__
    #print()
    
#temp_sm
#temp

In [ ]:
screener[['risk_trend_factor']].replace(['missing','error'], np.NaN).dropna().hist()

In [ ]:
len(screener)

In [ ]:
risk_screened = screener_good[(screener_good['risk_trend_factor']>risk_trend_threshold) & (screener_good['volume_factor']!= 'missing')]
risk_screened = risk_screened.merge(temp, on='Symbol', how='left')
print(risk_screened.columns)
display(risk_screened[['Symbol','latest_p','risk_trend_factor','adf','hurst','Sector','Market Index','vol_30d_2yr']])
#left join sector
#risk_screened[['risk_trend_factor']].dropna().hist()

In [ ]:
#screener.columns

In [ ]:
"""
above_1yearSDev = screener_good[screener_good['latest_p']>screener['mean_1y']+screener_good['std_1y']]['Symbol'].values
above_2yearSDev = screener_good[screener_good['latest_p']>screener['mean_2y']+screener_good['std_2y']]['Symbol'].values
above_yearSDev = np.intersect1d(above_1yearSDev,above_2yearSDev)
"""

In [ ]:
#above_yearSDev

In [ ]:
"""
above_2year_t = screener[screener['pct_2yr']>threshold]['Symbol'].values
above_1year_t = screener[screener['pct_1yr']>threshold]['Symbol'].values
above_years_t = np.intersect1d(above_2year_t,above_1year_t)
"""

In [ ]:
#screened_final_set = risk_screened.set_index('Symbol')
#screened_final_set
screened_final_set = risk_screened

In [ ]:
import pickle
fundamentals_quarterlies = pickle.load(open('./fundamental.pkl', 'rb'))

In [ ]:
fundamental_entries = [e[0] for e in fundamentals_quarterlies]

In [ ]:
list_ = []
for i in index_risk_sorted['Symbol'].values:
    list_.append(i)
for i in sectors_risk_sorted['Symbol'].values:
    list_.append(i)
for i in screened_final_set['Symbol'].values:
    list_.append(i)
indexes = np.unique(list_, return_index=True)[1]
list__ = [list_[index] for index in sorted(indexes)]

In [ ]:
#np.unique(stock_sectors.values)

In [ ]:
dict_sectors = {'Basic Materials':'VAW', 'Communication Services':'VOX', 'Consumer Cyclical':'VCR',
       'Consumer Defensive':'VDC', 'Energy':'VDE', 'Financial Services':'VFH', 'Healthcare':'VHT',
       'Industrials':'VIS', 'Real Estate':'VNQ', 'Technology':'VGT', 'Utilities':'VPU', 'error':'error'}

In [ ]:
dict_indexes = {'SPTM':'SP1500', 'SPY':'SP500', 'SLY':'SP600', 'MDYG':'SP400','error':'error'}

In [ ]:
dict_sectors_reverse = {'VAW':'Basic Materials',
 'VOX':'Communication Services',
 'VCR':'Consumer Cyclical',
 'VDC':'Consumer Defensive',
 'VDE':'Energy',
 'VFH':'Financial Services',
 'VHT':'Healthcare',
 'VIS':'Industrials',
 'VNQ':'Real Estate',
 'VGT':'Technology',
 'VPU':'Utilities',
 'error': 'error'}

In [ ]:
client = Client('192.168.3.100:8786')

tscv = expanding_window(initial =52*2, horizon = 13,period = 26)

batchClearLimit = round(len(list__)/7)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
 
decision_metrics = []

print("adf < .05 or .01, mean reverting")
print("Hurst")
print("> .5 - The time series is mean reverting.")
print("= .5 - The time series is a Geometric Brownian Motion.")
print("< .5 - The time series is trending.")
"""*spdr_indexes['Symbol'].values,*vanguard_sectors['Symbol'].values,"""
for s in list__:
#for s in ['CEIX']:
    msize = 2

    metrics_df_ = pd.DataFrame(pd.DataFrame(screener.loc[s].dropna()).T[['volume_factor','latest_p','vol_30d_2yr','pct_2yr','adf','hurst']])

    subset=prices[prices['Symbol']==s].reindex(nyse_dates.index).interpolate(method='time')

    dates = []
    for t in subset.index.values:
        d = pd.Timestamp(t).strftime('%Y-%m-%d')
        dates.append(d)
        dto = datetime.strptime(d, '%Y-%m-%d').date()

    old_ordinal = [datetime.strptime(i, '%Y-%m-%d').toordinal() for i in dates]
    new_ordinal = old_ordinal + md.date2num(np.datetime64('0000-12-31'))
    x = new_ordinal    

    lookup_index_ = []

    labels = list()
    print("symbol:",s)   

    try:
        symbol_sector = dict_sectors[sector_merged[sector_merged['Symbol']==s]['Sector'].values[0]]
        print("sector:",dict_sectors_reverse[symbol_sector])
    except:
        if(symbol_sector=='error'):
            if(np.sum([vanguard_sectors['Symbol'].values==s])):
                symbol_sector = s
                print("sector:",dict_sectors_reverse[symbol_sector])
        else:
            symbol_sector = 'error'
    if(symbol_sector=='error'):
        if(np.sum([vanguard_sectors['Symbol'].values==s])):
            symbol_sector = s
            print("sector:",dict_sectors_reverse[symbol_sector])
        else:
            symbol_sector = 'error'
            print("sector:",'error')
    """
    if(symbol_sector=='error'):
        try:
            dict_sectors[symbol_sector]
            symbol_sector = dict_sectors_reverse[s]
        except:
            pass
    """

    symbol_index = temp_sm.loc[s]['index_Symbol']

    try:
        matched_index_name = spdr_indexes.iloc[np.where(spdr_indexes['Symbol'].values==s)].index[0]
    except:
        try:
            matched_index_name = temp_sm.loc[s]['index']
        except:
            matched_index_name = "none"

    print("Index:",matched_index_name)

    print("supply trend:",screener.iloc[np.where(screener.index==s)]['vol_30d_2yr'][0])
    print("risk trend factor:",screener.iloc[np.where(screener.index==s)]['risk_trend_factor'][0])   

    temp = pd.DataFrame(stock_fundamentals.loc[s])
    print(temp[np.array(temp!='error')].replace([np.inf,'inf','error', -np.inf], np.nan).dropna().T)

    last_date = pd.DataFrame(subset.iloc[-1][['21dBOLD','21dMA-TP','21dBOLU']],index=[prices[prices['Symbol']==s].index[-1].strftime('%Y-%m-%d')]).index[0]
    #print(last_date)
    bbands = pd.DataFrame(subset.iloc[-1][['Adj Close','21dBOLD','21dMA-TP','21dBOLU','90dSMA','90dSDev']]).T
    bbands['90d_lower'] = bbands['90dSMA']-bbands['90dSDev']*2
    bbands['90d_upper'] = bbands['90dSMA']+bbands['90dSDev']*2
    
    #print(bbands)
    bbands.index = [last_date]
    display(bbands)

    display(metrics_df_)
    
    if(metrics_df_['adf'][0]<.05):
        print("ADFuller H0 rejected @ .05, Mean Reverting TS")
        if(bbands['Adj Close'][0]<bbands['90d_lower'][0]):
            colored_text = colored(255, 0, 0, "Quarter Mean Reverting buy signal")
            print(colored_text)
        if(bbands['Adj Close'][0]<bbands['21dBOLU'][0]):
            colored_text = colored(255, 0, 0, "Month Mean Reverting buy signal")
            print(colored_text)
    
    fig, ax1 = plt.subplots()   
    
    ax2 = ax1.twinx()
    
    l_axis_legend = pd.DataFrame(['symbol', 'trailing_1yr_max', 'trailing_1yr_min', 'risk_trend_factor','linear','supply_trend_1yr'],index=['blue','green','red','orange','black','yellow'],columns=['legend'])
    #display(l_axis_legend)
    r_axis_legend = pd.DataFrame(['sector','index'],index=['magenta','cyan'],columns=['legend'])
    #display(r_axis_legend)

    display_side_by_side([l_axis_legend, r_axis_legend], ['l axis', 'r axis'])
        
    #labels_ax2 = list()
    #if(np.sum(spdr_indexes['Symbol'].values==s)>0):    
    
    if(symbol_sector=='error'):
        pass
        """
        if(np.sum(vanguard_sectors['Symbol'].values==s)>0):
            #print("sector:",dict_sectors_reverse[s])
            ax2.plot(x, prices[prices['Symbol']==s]['Adj Close'],marker = '.',markersize=msize)
            #plt.plot(prices[prices['Symbol']==s]['Adj Close'])
            labels.append(dict_sectors_reverse[s])
            #plt.legend([dict_sectors_reverse[s]])
            #plt.show()
        elif((temp_sm.loc[s]['Sector']=='error')):
            #print("pass")
            pass
        """
    else:
        if(np.sum(vanguard_sectors['Symbol'].values==s)>0):
            pass
        else:
            #print("sector",)
            #print("else_sector:",dict_sectors_reverse[s])
            ax2.plot(x, prices[prices['Symbol']==symbol_sector]['Adj Close'],marker = '.',markersize=msize,color='m',linestyle=(0, (3, 10, 1, 10)))        
            #plt.legend([temp_sm.loc[s]['Sector']])
            #labels_ax2.append(temp_sm.loc[s]['Sector'])
            #plt.show()

    if(str(temp_sm.loc[s]['index_Symbol'])!='nan'):
        if(temp_sm.loc[s]['index_Symbol']!='error'):
            #print(temp_sm.loc[s]['index_Symbol'])
            ax2.plot(x, prices[prices['Symbol']==temp_sm.loc[s]['index_Symbol']]['Adj Close'],marker = '.',markersize=msize,color='c',linestyle=(0, (1, 10)))
            #plt.plot()
            #plt.legend([temp_sm.loc[s]['index']])
            #labels_ax2.append(temp_sm.loc[s]['index'])
            #plt.show()

    y= subset['Adj Close']

    slope, intercept, r, p, std_err = stats.linregress(x, y)

    mymodel = list(map(myfunc, x))

    ax1.plot(x, y,marker = '.',markersize=msize, color = 'b')
    ax1.plot(x, subset['trailing_1yr_max'],marker = '.',markersize=msize, color = 'g')
    ax1.plot(x, subset['trailing_1yr_min'],marker = '.',markersize=msize, color = 'r')
    ax1.plot(x, subset['risk_trend_factor'],marker = '.',markersize=msize, color = 'orange')
    
    
    #plt.plot(x, subset['21dBOLD'],marker = '.',markersize=msize)
    #plt.plot(x, subset['21dBOLU'],marker = '.',markersize=msize)
    #plt.plot(x, subset['21dMA-TP'],marker = '.',markersize=msize)
    #plt.plot(x, subset['90dSMA'],marker = '.',markersize=msize)
    #plt.plot(x, subset['90dSMA']+subset['90dSDev']*2,marker = '.',markersize=msize)
    #plt.plot(x, subset['90dSMA']-subset['90dSDev']*2,marker = '.',markersize=msize)
    
    ax1.plot(x, subset['supply_trend_1yr'],marker = '.',markersize=msize, color = 'y')
    
    ax1.plot(x, mymodel, color = 'k')
    
    #for l in ['1yrMax','1yrMin','21dBOLD','21dBOLU','21dMA-TP','90dSMA','90dSMA+Sdev*2','90dSMA-Sdev*2','supply_trend_1yr','linear']:
    for l in ['symbol','1yrMax','1yrMin','risk_trend_factor','supply_trend_1yr','linear']:
        labels.append(l)

    """
    if(lookup_sector_ == 'error'):
        labels.append(['1yrMax','1yrMin','21dBOLD','21dBOLU','21dMA-TP','90dSMA','90dSMA+Sdev*2','90dSMA-Sdev*2','supply_trend_1yr','linear'])
    else:
        labels = ['1yrMax','1yrMin','21dBOLD','21dBOLU','21dMA-TP','90dSMA','90dSMA+Sdev*2','90dSMA-Sdev*2','supply_trend_1yr','linear']
    """
    #ax1.legend([*labels,*labels_ax2], fontsize=8,loc=2)
    #ax1.legend(labels_ax2, fontsize=8,loc=2)
    
    l = matplotlib.dates.AutoDateLocator()
    f = matplotlib.dates.ConciseDateFormatter(l)    
    
    ax1.set_xticklabels(ax1.get_xticks(), rotation = 45)
    
    locator = matplotlib.dates.AutoDateLocator()
    formatter = matplotlib.dates.ConciseDateFormatter(locator)

    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    
    date_form = DateFormatter("%Y-%m-%d")
    ax1.xaxis.set_major_formatter(date_form)
    plt.show()
    
    print("ets next 13 weeks")
    
    #ets_LAdA=sm.tsa.statespace.ExponentialSmoothing(np.log(subset['Adj Close'].asfreq('D').interpolate().asfreq('W')),initialization_method='heuristic',seasonal=52,trend=True,damped_trend=True).fit()
    #ets_LAdA_ndt=sm.tsa.statespace.ExponentialSmoothing(np.log(subset['Adj Close'].asfreq('D').interpolate().asfreq('W')),initialization_method='heuristic',seasonal=52,trend=False,damped_trend=True).fit()

    newDates = pd.date_range(end, (end + dt.timedelta(92)).strftime('%Y-%m-%d'), freq='W').map(lambda t: t.strftime('%Y-%m-%d'))

    data = subset[['Adj Close']].asfreq('D').interpolate().asfreq('W')

    if(np.where(np.array(list__)==s)[0][0]%batchClearLimit==0):        
        client.restart()
        
    npa = []
    for train_index, test_index in tscv.split(data):

        train = data.iloc[train_index]
        test = data.iloc[test_index]

        npa.append([train,test])
    
    future = client.map(evaluate_ets, npa)
    results_rmses = []

    #my intent was to capture future objects vs results and this gave me results
    for f in as_completed(future):
        #exclude errors
        if(f.status=="error"):
            #pass
            results_rmses.append([np.nan,np.nan,np.nan,np.nan])
        else:
            results_rmses.append(f.result()) 

    rmse_TT = [r[0] for r in results_rmses].copy()
    rmse_FT = [r[1] for r in results_rmses].copy()
    rmse_TF = [r[2] for r in results_rmses].copy()
    rmse_FF = [r[3] for r in results_rmses].copy()

    #del results_rmses
    #del npa
        
    models = ['ets_TT','ets_FT','ets_TF','ets_FF']
    print(models)
    #scores = [np.mean(rmse_TT),np.mean(rmse_FT),np.mean(rmse_TF),np.mean(rmse_FF),np.mean(rmse_arima)]
    scores = [np.nanmean(rmse_TT),np.nanmean(rmse_FT),np.nanmean(rmse_TF),np.nanmean(rmse_FF)]
    sdevs = [np.nanstd(rmse_TT),np.nanstd(rmse_FT),np.nanstd(rmse_TF),np.nanstd(rmse_FF)]
    print(scores)

    best_model = models[np.argmin(scores)]

    print("best model:",best_model)
    print("best score:",scores[np.argmin(scores)])

    if(best_model=='ets_TT'):
        final_model=sm.tsa.statespace.ExponentialSmoothing(np.log(data),initialization_method='heuristic',seasonal=52,trend=True,damped_trend=True).fit()
    elif(best_model=='ets_FT'):
        final_model=sm.tsa.statespace.ExponentialSmoothing(np.log(data),initialization_method='heuristic',seasonal=52,trend=False,damped_trend=True).fit()
    elif(best_model=='ets_TF'):
        final_model=sm.tsa.statespace.ExponentialSmoothing(np.log(data),initialization_method='heuristic',seasonal=52,trend=True,damped_trend=False).fit()
    elif(best_model=='ets_FF'): 
        final_model=sm.tsa.statespace.ExponentialSmoothing(np.log(data),initialization_method='heuristic',seasonal=52,trend=False,damped_trend=False).fit()
    else:
        pass

    pred = final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d'))
    df_pred = np.exp(pred.summary_frame(alpha=0.05))
    s_date = df_pred.index[np.argmax(df_pred['mean'].values)].strftime('%Y-%m-%d')
    
    print("sell date:",s_date)
    e_return = np.array((np.max(df_pred['mean'].values)-metrics_df_['latest_p'])/metrics_df_['latest_p'])[0]
    
    print("expected return:",e_return)
        
    days_delta = (datetime.strptime(s_date, '%Y-%m-%d') - datetime.strptime(end.strftime('%Y-%m-%d'), '%Y-%m-%d')).days
    ext_return = (1+e_return)**(1/days_delta)-1
    
    #used to help filter at the end, not sdevs is chosen based on min score
    decision_metrics.append([s,e_return,s_date,days_delta,ext_return,scores[np.argmin(scores)],sdevs[np.argmin(scores)]])
    
    plt.plot(data.tail(13))
    plt.plot(df_pred[['mean','mean_ci_lower','mean_ci_upper']])
    plt.xticks([*data.tail(13).index.map(lambda t: t.strftime('%Y-%m-%d')),*newDates],rotation = 45,size=8)
    plt.show()

    #display(pd.DataFrame(stock_fundamentals.set_index(stock_fundamentals.iloc[:,0]).loc[s]).dropna().T)
    
    #evaluate_models(subset['Adj Close'].values, p_values, d_values, q_values)
    
    if(np.sum(np.array(fundamental_entries)==s)>0):
        loc = np.where(np.array(fundamental_entries)==s)[0][0]
        #print(fundamentals_quarterlies[loc[0]][0])
        #fundamentals_quarterlies[loc][1]
        for item in fundamentals_quarterlies[loc][1]:
            if(type(item[1]) == type(None)):
                pass
            else:
                if(len(item[1])>0):
                    #print(len(item[1]))
                    print(item)
client.close()

In [147]:
def highlight_cells_(val, color_if_true, color_if_false):
    color = color_if_true if val >= .75 else color_if_false
    return 'background-color: {}'.format(color)


SyntaxError: invalid syntax (2739505829.py, line 2)

In [148]:
decision_metrics_df = pd.DataFrame(decision_metrics,columns=['SYMBOL','e_return','s_date','days_delta','ext_return','error','sdev'])
decision_metrics_df = decision_metrics_df.set_index('SYMBOL').sort_values(by='ext_return',ascending=False)
decision_metrics_df['price'] = screener['latest_p'].loc[decision_metrics_df.index]
decision_metrics_df['qtr_return'] = (1+decision_metrics_df[['ext_return']])**92
decision_metrics_df['volatility'] = decision_metrics_df['error']/screener['latest_p'].loc[decision_metrics_df.index]
decision_metrics_df['volatility_sdev'] = decision_metrics_df['sdev']/screener['latest_p'].loc[decision_metrics_df.index]
#decision_metrics_df[decision_metrics_df['volatility']<=.75]
#decision_metrics_df[decision_metrics_df['volatility']>.75]
decision_metrics_df.style.applymap(highlight_cells_, color_if_true='yellow', color_if_false='#C6E2E9', 
                  subset=['volatility'])

,e_return,s_date,days_delta,ext_return,error,sdev,price,qtr_return,volatility,volatility_sdev
SYMBOL,,,,,,,,,,
VPU,0.035810,2022-06-12,1,0.035810,158.165968,246.644474,155.610001,25.455615,1.016425,1.585017
SM,0.732324,2022-09-11,92,0.005990,36.830435,24.573108,50.560001,1.732324,0.728450,0.486019
APA,0.117840,2022-07-03,22,0.005076,64.452728,76.256969,48.810001,1.593366,1.320482,1.562323
LNTH,0.411257,2022-09-11,92,0.003751,124.177717,268.614576,63.849998,1.411257,1.944835,4.206963
DVN,0.353998,2022-09-11,92,0.003300,59.807669,44.874308,74.059998,1.353998,0.807557,0.605918
OXY,0.302473,2022-09-11,92,0.002877,168.493923,190.109997,63.869999,1.302473,2.638076,2.976515
MRO,0.280787,2022-09-11,92,0.002694,15.201520,14.997104,30.290001,1.280787,0.501866,0.495117
CTRA,0.058957,2022-07-03,22,0.002607,8.613657,4.018646,34.310001,1.270679,0.251054,0.117128
RRC,0.265421,2022-09-11,92,0.002562,9.614881,14.729786,34.439999,1.265421,0.279178,0.427694
